<img src="https://www.lsce.ipsl.fr/Css/img/banniere_LSCE_75.png" width="600px">

## Mapping DYNAMICO output with geovista

- Author: Patrick Brockmann
- Version: 22/05/2024

In [32]:
import pyvista as pv
import geovista as gv
import xarray as xr

ds = xr.open_dataset("https://thredds-su.ipsl.fr/thredds/dodsC/ipsl_thredds/brocksce/ICO/ICO.79.1jour.native.1_19790101_19790101_1D_inca_ges.nc")

blon = ds['bounds_lon'].to_numpy()
blat = ds['bounds_lat'].to_numpy()
var = ds['temp'][0,0]

# Read https://github.com/bjlittle/geovista/issues/202
mesh = gv.Transform.from_unstructured(blon, blat, name='Data', data=var.to_numpy())
mesh['Data'] = var.to_numpy()

pv.global_theme.lighting_params.ambient = 0.3

#pl = gv.GeoPlotter(crs="+proj=robin lon_0=180")
pl = gv.GeoPlotter()

sargs = {'title': f'{var.long_name} / {var.units}'}
pl.add_mesh(mesh, show_edges=False, scalar_bar_args=sargs, cmap='plasma')
pl.add_coastlines(color='black')
pl.add_axes()

pl.show()

Widget(value='<iframe src="http://localhost:40449/index.html?ui=P_0x7fcf76d71460_15&reconnect=auto" class="pyv…

### Add widgets for more interactivity

In [24]:
ds = xr.open_dataset("https://thredds-su.ipsl.fr/thredds/dodsC/ipsl_thredds/brocksce/ICO/ICO.79.1jour.native.1_19790101_19790101_1D_inca_ges.nc")
ds

<xarray.Dataset> Size: 3GB
Dimensions:               (cell: 16002, nvertex: 6, presnivs: 79,
                           time_counter: 24, axis_nbounds: 2)
Coordinates:
    lat                   (cell) float32 64kB ...
    lon                   (cell) float32 64kB ...
  * presnivs              (presnivs) float32 316B 1.012e+05 1.009e+05 ... 1.483
    time_centered         (time_counter) datetime64[ns] 192B ...
  * time_counter          (time_counter) datetime64[ns] 192B 1979-01-01T00:30...
    time_instant          (time_counter) datetime64[ns] 192B ...
Dimensions without coordinates: cell, nvertex, axis_nbounds
Data variables: (12/29)
    bounds_lon            (cell, nvertex) float32 384kB ...
    bounds_lat            (cell, nvertex) float32 384kB ...
    time_centered_bounds  (time_counter, axis_nbounds) datetime64[ns] 384B ...
    time_counter_bounds   (time_counter, axis_nbounds) datetime64[ns] 384B ...
    time_instant_bounds   (time_counter, axis_nbounds) datetime64[ns] 384B ...
    pmid                  (time_counter, presnivs, cell) float64 243MB ...
    ...                    ...
    vmrmcf                (time_counter, presnivs, cell) float64 243MB ...
    vmrbe7                (time_counter, presnivs, cell) float64 243MB ...
    vmrn2o                (time_counter, presnivs, cell) float64 243MB ...
    vmrch4                (time_counter, presnivs, cell) float64 243MB ...
    vmrco                 (time_counter, presnivs, cell) float64 243MB ...
    vmrco2bih             (time_counter, presnivs, cell) float64 243MB ...
Attributes:
    _NCProperties:                   version=1|netcdflibversion=4.6.0|hdf5lib...
    name:                            inca1d_ges
    description:                     Created by xios
    title:                           Created by xios
    Conventions:                     CF-1.6
    timeStamp:                       2022-Apr-05 08:59:15 GMT
    uuid:                            8982bdb7-b650-45c4-896c-e228bec0cb5a
    history:                         Fri Apr 15 12:05:37 2022: ncatted -O -a ...
    NCO:                             4.6.9
    DODS_EXTRA.Unlimited_Dimension:  time_counter

In [25]:
blon = ds['bounds_lon'].to_numpy()
blat = ds['bounds_lat'].to_numpy()

mesh = gv.Transform.from_unstructured(blon, blat, name='Data')
mesh

PolyData (0x7fcf24c18580)
  N Cells:    16002
  N Points:   96012
  N Strips:   0
  X Bounds:   -1.000e+00, 1.000e+00
  Y Bounds:   -9.998e-01, 9.998e-01
  Z Bounds:   -9.999e-01, 9.999e-01
  N Arrays:   2

In [26]:
print(set(ds.dims))
print(set(ds.coords))
# Coordinates variables (variable name = dimension name) indicated by a * in xarray.Dataset
print(list(set(ds.dims) & set(ds.coords)))

{'time_counter', 'axis_nbounds', 'nvertex', 'cell', 'presnivs'}
{'time_counter', 'time_instant', 'time_centered', 'lat', 'presnivs', 'lon'}
['time_counter', 'presnivs']


In [27]:
# Could be detected automatically
dimK = 'presnivs'
dimL = 'time_counter'

In [28]:
kmax = ds[dimK].shape[0]
lmax = ds[dimL].shape[0]
print(kmax, lmax)

79 24


In [29]:
ds.data_vars

Data variables:
    bounds_lon            (cell, nvertex) float32 384kB 36.0 53.45 ... 144.0
    bounds_lat            (cell, nvertex) float32 384kB 86.6 87.07 ... -89.12
    time_centered_bounds  (time_counter, axis_nbounds) datetime64[ns] 384B ...
    time_counter_bounds   (time_counter, axis_nbounds) datetime64[ns] 384B ...
    time_instant_bounds   (time_counter, axis_nbounds) datetime64[ns] 384B ...
    pmid                  (time_counter, presnivs, cell) float64 243MB ...
    sh                    (time_counter, presnivs, cell) float64 243MB ...
    pdel                  (time_counter, presnivs, cell) float64 243MB ...
    zdens                 (time_counter, presnivs, cell) float64 243MB ...
    temp                  (time_counter, presnivs, cell) float64 243MB ...
    AIRMASS               (time_counter, presnivs, cell) float64 243MB ...
    ps                    (time_counter, cell) float64 3MB ...
    area                  (time_counter, cell) float64 3MB ...
    emirn222    

In [30]:
def get_vars():
    vars = {}
    for var in ds.data_vars:
        dims = list(ds[var].dims)
        if dims[-1] == 'cell':
            dims.pop(-1)
            ndims = [d.replace(dimL, 'L').replace(dimK, 'K') for d in dims]
            vars[var] = {'dims': ''.join(ndims)}
    return vars

def get_title():
    try:
        title = ds[variable].name + ' (' + ds[variable].long_name + ')'
    except:
        title = ds[variable].name
    return title

def get_units():
    try:
        units = ds[variable].units
    except:
        units = ''
    return units + '\n'

In [31]:
import matplotlib
import numpy as np
import geovista as gv
import pyvista as pv
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox

#==================================
variable = 'temp'
kindex = 1
lindex = 1
var_info = ""

vars = get_vars()
cmap = 'plasma'
auto_range = True
show_edges = False

widget_edges = widgets.Checkbox(value=show_edges, description='Edges')
widget_k = widgets.IntSlider(min=1, max=kmax, step=1, description='K :')
widget_l = widgets.IntSlider(min=1, max=lmax, step=1, description='L :')
widget_range = widgets.Checkbox(value=auto_range, description='Range update')
widget_cmap = widgets.Dropdown(options=matplotlib.colormaps(), value=cmap, description='cmap :')
widget_var = widgets.Dropdown(options=sorted(vars.keys()), value=variable, description='variable :')

#==================================
def load_var():
    global var, var_info
    if vars[variable]['dims'] == 'LK':
        widget_k.disabled = False
        widget_l.disabled = False
        var = ds[variable][lindex-1, kindex-1]
        try:
            zlevels_units = '[' + ds['presnivs'].units + ']'
        except:
            zlevels_units = ''
        var_info = "K=%02d/%02d Z=%s %s" %(kindex, kmax, ds['presnivs'][kindex-1].values, zlevels_units)
        var_info += "\nL=%03d/%03d T=%s" %(lindex, lmax, ds['time_counter'][lindex-1].values)
    elif vars[variable]['dims'] == 'L':
        widget_k.disabled = True
        widget_l.disabled = False
        var = ds[variable][lindex-1]
        var_info = "L=%03d/%03d T=%s" %(lindex, lmax, ds['time_counter'][lindex-1].values)
    elif vars[variable]['dims'] == 'K':
        widget_k.disabled = False
        widget_l.disabled = True
        var = ds[variable][kindex-1]
        try:
            zlevels_units = '[' + ds['presnivs'].units + ']'
        except:
            zlevels_units = ''
        var_info = "K=%03d/%03d Z=%s %s" %(kindex, kmax, ds['presnivs'][kindex-1].values, zlevels_units)     
    elif vars[variable]['dims'] == '':
        widget_k.disabled = True
        widget_l.disabled = True
        var = ds[variable]
        var_info = ""
    varmin = np.min(var.values)
    varmax = np.max(var.values)
    varmean = np.mean(var.values)
    stat_text = "Min: %e\nMax: %e\nMean: %e" %(varmin, varmax, varmean)
    var_info = var_info + '\n' + stat_text
    
    return var

var = load_var()

#==================================
varmin = np.min(var.values)
varmax = np.max(var.values)
cmap_name = 'plasma'
clim = [varmin, varmax]

#==================================
pv.global_theme.lighting_params.ambient = 0.3

pl = gv.GeoPlotter()
pl.add_coastlines(color='black')

var = ds[variable][lindex, kindex]
mesh['Data'] = var

#==================================
mesh_actor = pl.add_mesh(
    mesh, 
    name = 'Data', 
    show_edges = show_edges,
    edge_color = 'gray',
    clim = clim,
    cmap = cmap_name, 
    show_scalar_bar = False
)

scalarBar_actor = pl.add_scalar_bar(
    get_units(),
    vertical = True,
    width = 0.05,
    height = 0.5,
    position_y = 0.3,
    title_font_size = 14,
    label_font_size = 12
)

title_actor = pl.add_text(
    get_title() + '\n' + var_info,
    position = 'lower_left', 
    font_size = 8
)

#==================================
def on_edges_change(e):
    show_edges = e.new
    mesh_actor.GetProperty().show_edges = show_edges
    pl.update()

#==================================
def on_kvalue_change(e):
    global kindex, var
    kindex = e.new
    load_var()
    mesh['Data'] = var
    title_actor.set_text(text=get_title() + '\n' + var_info, position='lower_left')
    pl.update()
    if auto_range: on_range_change(e)

#==================================
def on_lvalue_change(e):
    global lindex, var
    lindex = e.new
    load_var()
    mesh['Data'] = var
    title_actor.set_text(text=get_title() + '\n' + var_info, position='lower_left')
    pl.update()
    if auto_range: on_range_change(e)

#==================================
def on_range_change(e):
    global auto_range
    auto_range = e.new
    varmin = np.min(var.values)
    varmax = np.max(var.values)
    clim = [varmin, varmax]
    mesh_actor.mapper.scalar_range = clim
    pl.update()

#==================================
def on_var_change(e):
    global variable, var
    variable = e.new
    load_var()
    mesh['Data'] = var
    title_actor.set_text(text=get_title() + '\n' + var_info, position='lower_left')
    scalarBar_actor.SetTitle(get_units())
    pl.update()
    if auto_range: on_range_change(e)

#==================================
def on_cmap_change(e):
    global cmap_name
    cmap_name = e.new
    mesh_actor.mapper.lookup_table.cmap = cmap_name
    pl.update()

#==================================
widget_var.observe(on_var_change, 'value')
widget_cmap.observe(on_cmap_change, 'value')
widget_k.observe(on_kvalue_change, 'value')
widget_l.observe(on_lvalue_change, 'value')
widget_range.observe(on_range_change, 'value')
widget_edges.observe(on_edges_change, 'value')

display( widgets.HBox([widgets.VBox([widget_var, widget_cmap]),
                       widgets.VBox([widget_k, widget_l]), 
                       widgets.VBox([widget_range, widget_edges])]) )

pl.show()

Widget(value='<iframe src="http://localhost:40449/index.html?ui=P_0x7fcf4c18ca40_14&reconnect=auto" class="pyv…